The purpose of this notebook is to identify the frequency of conflicts between metrics that purport to measure the degree to which a plan favors one party or another.

In [1]:
from typing import List, Dict, Any, Set

import os
import pandas as pd
from collections import defaultdict

from rdametrics import states, chambers, ensembles

Load the scores dataframe

In [2]:
scores_path: str = "~/local/beta-ensembles/prepackaged/scores/scores.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))

Helper code

In [7]:
# from typing import Union
import numpy as np
import pandas as pd

ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

partisan_bias = {
    "partisan_advantage": ["disproportionality", "efficiency_gap"],
    "symmetry": ["geometric_seats_bias", "seats_bias", "votes_bias"],
    "packing_cracking": ["mean_median_average_district", "lopsided_outcomes", "declination"],
}

categories: List[str] = list(partisan_bias.keys())

# def same_sign(a: float | int | None, b: float | int | None) -> bool:
#     if a == 0.0 or b == 0.0:
#         return True
    
#     if pd.isna(a) or pd.isna(b):
#         return True

#     return a * b > 0

def same_sign(a, b):
    if a == 0.0 or b == 0.0:
        return True
    
    if pd.isna(a) or pd.isna(b):
        return True
    
    assert isinstance(a, (int, float)) and isinstance(b, (int, float)), f"Unexpected types: {type(a)}, {type(b)}"

    return a * b > 0

# def same_signs(list1, list2):
#     # Combine both lists and filter out zero values and NaN values
#     all_values = list1 + list2
#     non_zero_defined_values = [x for x in all_values if x != 0 and not pd.isna(x)]
    
#     # If no valid values or only one valid value, return True
#     if len(non_zero_defined_values) <= 1:
#         return True
    
#     # Check if all valid values have the same sign
#     all_positive = all(x > 0 for x in non_zero_defined_values)
#     all_negative = all(x < 0 for x in non_zero_defined_values)
    
#     return all_positive or all_negative



Setup counters for various kinds of conflicts.

In [ ]:
from itertools import combinations
import copy
from collections import defaultdict

by_state: Dict[str, Any] = dict()

for xx in states:
    by_state[xx] = {
        "Vf": None,
        "total": 0
    }

conflicts = dict()
_ledger = {
    "counts": defaultdict(int),
    "combos": set(),
    "example": None,
    "value1": None,
    "value2": None,
    "delta": None,
}

for c in categories:
    conflicts[c] = dict()
    pairs = combinations(partisan_bias[c], 2)
    for p in pairs:
        conflicts[c][p] = copy.deepcopy(_ledger)

conflicts["cross_category"] = dict()
pairs = combinations(categories, 2)
for p in pairs:
    conflicts["cross_category"][p] = copy.deepcopy(_ledger)

# conflicts

Count instances where the scores conflict. Keep track by state / chamber / ensemble combination.

In [ ]:
for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)

    by_state[xx]["total"] += 1
    if by_state[xx]["Vf"] is None:
        by_state[xx]["Vf"] = row["estimated_vote_pct"]

    partisan_advantage_consistent: bool = True
    packing_cracking_consistent: bool = True
    partisan_symmetry_consistent: bool = True

    # Check consistency within each category

    for c in categories:
        for _pair, _ledger in conflicts[c].items():
            m1, m2 = _pair
            v1 = row[m1]
            v2 = row[m2]
            delta = abs(v1 - v2)
            if not same_sign(v1, v2):
                _ledger["counts"][xx] += 1
                _ledger["combos"].add(combo)
                if _ledger["example"] is None or delta > _ledger["delta"]:
                    _ledger["example"] = combo
                    _ledger["value1"] = v1
                    _ledger["value2"] = v2
                    _ledger["delta"] = delta

    # Compare consistency across categories


# (by_state, conflicts)

({'FL': {'Vf': 0.4837, 'total': 659997},
  'IL': {'Vf': 0.5817, 'total': 659997},
  'MI': {'Vf': 0.5188, 'total': 659997},
  'NC': {'Vf': 0.4943, 'total': 659997},
  'NY': {'Vf': 0.6478, 'total': 659997},
  'OH': {'Vf': 0.4638, 'total': 659997},
  'WI': {'Vf': 0.5068, 'total': 659997}},
 {'partisan_advantage': {('disproportionality',
    'efficiency_gap'): {'counts': defaultdict(int,
                {'FL': 19389,
                 'IL': 473093,
                 'MI': 13610,
                 'NC': 6854,
                 'NY': 160598,
                 'OH': 4880,
                 'WI': 2323}), 'combos': {('FL', 'congress', 'A0'),
     ('FL', 'congress', 'B'),
     ('FL', 'congress', 'C'),
     ('FL', 'congress', 'D'),
     ('FL', 'congress', 'Pop+'),
     ('FL', 'congress', 'Pop-'),
     ('FL', 'congress', 'R100'),
     ('FL', 'congress', 'R25'),
     ('FL', 'congress', 'R50'),
     ('FL', 'congress', 'R75'),
     ('FL', 'congress', 'Rev'),
     ('FL', 'lower', 'A0'),
     ('FL', 'lower',